In [ ]:
#Importing the libraries needed

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")


In [ ]:
from imblearn.over_sampling import RandomOverSampler
sns.set(style='whitegrid')

In [ ]:
train  = pd.read_csv("../input/health-insurance-cross-sell-prediction/train.csv")
test = pd.read_csv("../input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
print("Shape of train dataset ",train.shape )
print("Shape of test dataset ",test.shape )

In [ ]:
#checking for null values

train.isnull().sum()

#we founnd that there are no null values in the dataset

In [ ]:
train.isnull().values.any()

In [ ]:
train = train.dropna(how='any',axis=0)

In [ ]:
train.info()

In [ ]:
#Concatinating both training and testing file for analysis

data= pd.concat([train,test],axis=0,sort=False)

In [ ]:
data.nunique()

In [ ]:
#Code to find numeric data
numeric_data = train.select_dtypes(include = np.number)
numeric_col = numeric_data.columns
numeric_data.head(1)

In [ ]:
#code to find categorical data
categorical_data = train.select_dtypes(exclude= np.number)
categorical_col = categorical_data.columns
categorical_data.head(1)

In [ ]:
a = train['Response'].value_counts()/len(train)*100
b = train['Gender'].value_counts()
c = train['Vehicle_Age'].value_counts()
d = train['Vehicle_Damage'].value_counts()

print(a)
print('\n')
print(b)
print('\n')
print(c)
print('\n')
print(d)




In [ ]:
#Outlier Analysis

def outliers(df, variable, distance):
     IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
     lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
     upper_boundary = df[variable].quantile(0.75) + (IQR * distance) 
        
     return upper_boundary, lower_boundary
    
upper_limit, lower_limit = outliers(train, 'Annual_Premium', 1.5)
upper_limit, lower_limit  

In [ ]:
data['Annual_Premium']= np.where(data['Annual_Premium'] > upper_limit, upper_limit, 
                                 np.where(data['Annual_Premium'] < lower_limit, lower_limit, data['Annual_Premium']))

In [ ]:
plt.hist(data['Annual_Premium'],bins=30);

**Exploratory** **data analysis**

In [ ]:
train.columns

In [ ]:
#Split the data set into numerical and categorical variables
numerical_columns = ['Age', 'Region_Code', 'Annual_Premium', 'Vintage']
categorical_columns =['Gender', 'Driving_License', 'Previously_Insured','Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Response' ] 

In [ ]:
train[numerical_columns].describe()

#The maximum age given is 85
#50% people are at the age of 36
#mean is 38.8
#standard deviation is 15.5

In [ ]:
col = train.columns.tolist()
col.remove('id')
train[col].describe(percentiles = [.25,.50,.75,.95,.99])

In [ ]:
# In the Annual_Premium the 99th percentile is 72544 and the max is 540165 this represents the outliers in this column. Hence it is an outlier

In [ ]:
#Here we have Response as the Independent varaiable or the target variable

In [ ]:
plt.subplot(1,2,1)
sns.countplot(train['Response'],palette = 'cool')
plt.title("Target variable count")



plt.subplot(1,2,2)
count = train['Response'].value_counts()
count.plot.pie(    autopct = '%1.1f%%',   colors=['pink', 'green'], figsize= (10,7), explode= [0,0.1], title = "pie chart of percentage of target class")

From the diagram we read that it is an imbalanced binary problem
Only 12.3 % customers are interested

In [ ]:
print( "Percentage of target class\n")
print(train['Response'].value_counts()/len(train)*100)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
train.hist(bins= 50, figsize= (10,10))
plt.show()

Plot on Gender

In [ ]:
plt.figure(figsize= (9,4))
plt.subplot(1,2,1)
sns.countplot(train['Gender'])
plt.title("Count of male and female")



plt.subplot(1,2,2)
sns.countplot(train['Gender'], hue= train['Response'], palette= "rocket_r")
plt.title("Response in male and female category")
plt.show()


In [ ]:
train['Gender'].value_counts()

In [ ]:
#There are 30619 males and 26223 females

#The chance that male would buy the insurance is high than females

EDA on Age

In [ ]:
plt.figure(figsize=(20,4))
sns.countplot(train['Age'], palette= 'hsv')
plt.title("count on age")
plt.show()

In [ ]:
plt.figure(figsize= (15,4))
plt.subplot(1,2,1)
sns.distplot(train['Age'])


plt.subplot(1,2,2)
sns.boxplot(train['Age'], palette= 'Accent')

EDA on Driving License

In [ ]:
plt.figure(figsize= (9,4))
plt.subplot(1,2,1)
sns.countplot(train['Driving_License'])


plt.subplot(1,2,2)
sns.countplot(train['Driving_License'], hue= train['Response'], data= train, palette= "rocket_r")

print("Percentage of  Driving_License feature\n ")
print(train['Driving_License'].value_counts()/len(train)*100)

In [ ]:
#Customers who have the DL are 99%
#Among people has DL who need insurance is less compared to who require no insurance

Region distribution

In [ ]:
plt.figure(figsize=(26,20))
plt.subplot(3,1,1)
sns.countplot(train["Region_Code"], data= train, palette= 'hsv')


plt.subplot(3,1,2)
sns.distplot(train['Region_Code'])
plt.title('Distribution of Region code')


plt.subplot(3,1,3)
sns.boxplot(train['Region_Code'])


In [ ]:
#Customers from Region Code 28 has more chance of buying insurance

EDA on Previously Insured

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.countplot(train['Previously_Insured'])

plt.subplot(1,2,2)
sns.countplot(train['Previously_Insured'], hue=train['Response'], palette= 'hsv' )

print("Percentage of Previously_Insured feature ")
print(train['Previously_Insured'].value_counts()/len(train)*100)

In [ ]:
#Cutomers who has not previously insured are likely to be interested

Vehicle Age analysis

In [ ]:
#plt.figure(figsize =(9,4))
#plt.subplot(1,2,1)
#sns.countplot(train['Vehicle_Age'])


plt.figure(figsize =(9,4))
plt.subplot(1,2,1)
ax = sns.countplot(train['Vehicle_Age'], data=train,
                   facecolor=(0, 0, 0, 0),
                   linewidth=5,
                   edgecolor=sns.color_palette("dark", 7))

plt.subplot(1,2,2)
sns.countplot(train['Vehicle_Age'], hue=train['Response'], facecolor=(0, 0, 0, 0),
                   linewidth=5, edgecolor=sns.color_palette("dark", 7))

In [ ]:
#There are less number of customers with vehicle age greater than 2 years
#customers with vehicle age 1-2 years are more likely to buy the insurance compared to other 

In [ ]:
plt.figure(figsize=(7,7))
train['Vehicle_Age'].value_counts().plot.pie(autopct='%1.1f%%', colors = ['r', 'b', 'g'])

In [ ]:
#Damage Vehicle and Response

plt.figure(figsize= (9,4))
plt.subplot(1,2,1)
sns.countplot(train['Vehicle_Damage'])


plt.subplot(1,2,2)
sns.countplot(train['Vehicle_Damage'], hue= train['Response'], palette ='binary_r')

In [ ]:
#Customers with vehicle damage are equallly distributed
#Customers with vehicle damage are more interested in the insurance

Annual Premiums

In [ ]:
plt.figure(figsize =(11,6))
plt.subplot (2,1,1)
sns.distplot(train['Annual_Premium'])
plt.show()

plt.subplot(2,1,2)
sns.boxplot(train['Annual_Premium'])
plt.show()

In [ ]:
#There are more outliers in the annual premium feature

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Correlation plot")
sns.heatmap(train.corr(),linewidths=5, annot=True, square=True,annot_kws={'size': 10},cmap='YlGnBu')

In [ ]:
#policy sales channel has slightly low correlation with Age variable

In [ ]:
train.head(5)

In [ ]:
#Age vs Annual Premium

In [ ]:
sns.scatterplot(x= train['Age'], y= train['Annual_Premium'])

In [ ]:
df =train.groupby(['Gender'])['Driving_License'].count().to_frame().reset_index()
df

In [ ]:
sns.catplot(x='Gender', y='Driving_License', data=df, kind='bar')

In [ ]:
#Vintage= Number of Days, Customer has been associated with the company

In [ ]:
sns.distplot(train.Vintage)

# Data Preprocessing

In [ ]:
train.columns

In [ ]:
train.head(5)

In [ ]:
train.columns

In [ ]:
train['Gender'] = train['Gender'].map( {'Female': 0, 'Male': 1} ).astype(int)

In [ ]:
train=pd.get_dummies(train,drop_first=True)

In [ ]:
train.head(5)

In [ ]:
#Renaming the column name

In [ ]:
train = train.rename(columns ={"Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1 Year", "Vehicle_Age_> 2 Years" : "Vehicle_Age_gt_ 2 Years"})

In [ ]:
train.head(5)

In [ ]:
num_feat = ['Age','Vintage']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#Apply Standardscaler for Age and Vintage
ss = StandardScaler()
train[num_feat]= ss.fit_transform(train[num_feat])

#Apply minmaxscaler for annual premium
mm=MinMaxScaler()
train['Annual_Premium']= mm.fit_transform(train[['Annual_Premium']])

In [ ]:
train.head(5)

For Test Data

In [ ]:
test['Gender'] = test['Gender'].map( {'Female': 0, 'Male': 1} ).astype(int)
test=pd.get_dummies(test,drop_first=True)
test=test.rename(columns={"Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year", "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"})
test['Vehicle_Age_lt_1_Year']=test['Vehicle_Age_lt_1_Year'].astype('int')
test['Vehicle_Age_gt_2_Years']=test['Vehicle_Age_gt_2_Years'].astype('int')
test['Vehicle_Damage_Yes']=test['Vehicle_Damage_Yes'].astype('int')

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
ss = StandardScaler()
test[num_feat] = ss.fit_transform(test[num_feat])

mm = MinMaxScaler()
test[['Annual_Premium']] = mm.fit_transform(test[['Annual_Premium']])

In [ ]:
train.head()

In [ ]:
test.head()

Splitting Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_target=train['Response']
train=train.drop(['Response'], axis = 1)

X_train,X_test,y_train,y_test = train_test_split(train,train_target, random_state = 0)

Modelling

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train,train_target, test_size=0.3, random_state= 124)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
model= DecisionTreeClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print (confusion_matrix(y_test, prediction))

In [ ]:
print (classification_report (y_test, prediction))

Random Forest Classifier

In [ ]:
from sklearn.ensemble.forest import RandomForestClassifier

In [ ]:
rf_model =RandomForestClassifier(n_estimators =150)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_prediction = rf_model.predict(X_test)

In [ ]:
print(classification_report(y_test, rf_prediction))

In [ ]:
print(confusion_matrix(y_test, rf_prediction))

In [ ]:
#Feature Importance

In [ ]:
feat_importances = pd.Series(rf_model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(15).plot(kind='barh')
#feat_importances.nsmallest(20).plot(kind='barh')
plt.show()

In [ ]:
#Hyper parameter tuning

In [ ]:
#LGBM CLASSIFIER

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
model=LGBMClassifier(colsample_bytree=0.5, learning_rate=0.03,
                     n_estimators=600, objective='binary', reg_alpha=0.1,
                     random_state=101,reg_lambda=0.8)

model.fit(X_train,y_train)

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
scores=cross_val_score(model, X_test, y_test, cv=cv,scoring='roc_auc')
scores.mean()

In [ ]:
val_pred= model.predict_proba(X_test)[:,1]

In [ ]:
val_pred

In [ ]:
print(roc_auc_score(y_test,val_pred))

LOGISTIC REGRESSION

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model


In [ ]:
model.fit (X_train, y_train)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print (classification_report (y_test, prediction))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
cm = confusion_matrix(y_test, prediction)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':11}, cmap = 'PuBu',fmt=".1f");

In [ ]:
#Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)
nb_model

In [ ]:
y_pred_nb = nb_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_nb)

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_nb)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
print(classification_report(y_test, y_pred_nb))

In [ ]:
#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
knn_model

In [ ]:
y_pred_knn = knn_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_knn)

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_knn)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
print(classification_report(y_test, y_pred_knn))

In [ ]:
#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier().fit(X_train, y_train)


In [ ]:
y_pred_rf = rf_model.predict(X_test)
accuracy_score(y_test, y_pred_rf)

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
Importance = pd.DataFrame({"Importance": rf_model.feature_importances_*100},
                         index = X_train.columns)

In [ ]:
Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "r")

plt.xlabel("Variable Significance Levels")

In [ ]:
#Gradient Boosting Machines

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbm_model = GradientBoostingClassifier().fit(X_train, y_train)

In [ ]:
y_pred_gbm = gbm_model.predict(X_test)
accuracy_score(y_test, y_pred_gbm)

In [ ]:
print(classification_report(y_test, y_pred_gbm))

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_gbm)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
#XG Boost

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
xgb_model = XGBClassifier().fit(X_train, y_train)

In [ ]:
y_pred_xgb_model = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred_xgb_model)

In [ ]:
print(classification_report(y_test, y_pred_xgb_model))

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_xgb_model)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
from xgboost import XGBClassifier
# Model Tuning
XGB_model = XGBClassifier(random_state = 42, max_depth = 8, n_estimators = 3000, 
                          reg_lambda = 1.2, reg_alpha = 1.2, 
                          min_child_weight = 1,objective = 'binary:logistic',
                         learning_rate = 0.15, gamma = 0.3, colsample_bytree = 0.5,
                          eval_metric = 'auc').fit(X_train, y_train)

In [ ]:
y_pred_XGB_model = XGB_model.predict(X_test)
accuracy_score(y_test, y_pred_XGB_model)

In [ ]:
print(classification_report(y_test, y_pred_XGB_model))

In [ ]:
# Cofusion Matrix
cm = confusion_matrix(y_test, y_pred_XGB_model)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu',fmt=".1f")

In [ ]:
models = [
    knn_model,
    nb_model,
    gbm_model,
    xgb_model,
    XGB_model,
    rf_model,
      
]

for model in models:
    names = model.__class__.__name__
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("-"*28)
    print(names + ":" )
    print("Accuracy: {:.4%}".format(accuracy))

In [ ]:
# Create submission file
submmission = pd.DataFrame()
submmission['id'] = test['id'].values
submmission['Response'] = y_test
submmission.to_csv('cat_submitfinal.csv', float_format='%.6f', index=False)

submmission.head()